In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import os
import numpy as np
import tifffile as tiff
import matplotlib.pyplot as plt
from skimage import measure
from scipy.spatial.transform import Rotation as R
from scipy.ndimage import map_coordinates
from scipy.optimize import minimize
from sklearn.decomposition import PCA
from tqdm import tqdm

In [17]:
#la première étape de notre projet consiste à prendre une image de cerveau de mouche et a determiner dans quel orientation il se situe 
#dans lespace à l'aide de quaternion. Pour cela nous avons recours à une PCA (principal component analysis) pour determiner les plus 
#grand axes du cerveau.  

In [18]:
def rotate_image_along_longest_axes(image):
    # Step 1: Extract coordinates of non-zero voxels
    activated_coords = np.array(np.where(image > 0)).T

    # Step 2: Perform PCA
    pca = PCA(n_components=3)
    pca.fit(activated_coords)

    # Get the principal components
    principal_components = pca.components_

    # Step 3: Use the principal components to define a rotation quaternion
    rotation_matrix = principal_components.T
    rotation_quaternion = R.from_matrix(rotation_matrix).as_quat()

    return rotation_quaternion


In [21]:
if __name__ == "__main__":

    input_dir = "C:/Users/Aurélien Delille/OneDrive - Saint Louis de Gonzague/Documents/epfl/project_bachelor/image_raw/source"
    # iterate over all the images in a given folder
    OUTPUT_DIR = "C:/Users/Aurélien Delille/OneDrive - Saint Louis de Gonzague/Documents/epfl/project_bachelor/image_rotated/rotated_source"

    if os.path.exists(OUTPUT_DIR):
        print(f" Directory {OUTPUT_DIR} already exists!")
        # otherwise mkdir
    else:
        os.makedirs(OUTPUT_DIR)
    if not os.path.exists(input_dir):
        print(f"Le dossier spécifié n'existe pas : {input_dir}")

    for file in tqdm(os.listdir(input_dir)):
        if file.endswith(".tif"):
            # check if fly already done if fly id exists in one folder name
            INPUT_PATH = os.path.join(input_dir, file)
            fly_id = INPUT_PATH.split('/')[-1].split('.')[0]
            print(f"Orienting fly: {fly_id}")
            # check if the file persistence_entropy_{fly_id}.csv exists in the
            # folder
            # /home/samuel/brainMorpho/Analysis_results/homology2voxel_pairwise/
            # if it exists, skip the fly
            filename = f"rotated_{fly_id}.tif"

            # check if output dir exists
            
            if os.path.exists(os.path.join(OUTPUT_DIR,filename)):
                pass
            original_image = tiff.imread(INPUT_PATH)

            quaternion = rotate_image_along_longest_axes(padded_image)

 Directory C:/Users/Aurélien Delille/OneDrive - Saint Louis de Gonzague/Documents/epfl/project_bachelor/image_rotated/rotated_source already exists!
Le dossier spécifié n'existe pas : C:/Users/Aurélien Delille/OneDrive - Saint Louis de Gonzague/Documents/epfl/project_bachelor/image_raw/source


FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/Aurélien Delille/OneDrive - Saint Louis de Gonzague/Documents/epfl/project_bachelor/image_raw/source'

In [ ]:
#On définit ensuite notre classe et notre modèle

In [2]:
class fly_alignement(nn.Module):
    def __init__(self, num_classes):
        super(fly_alignement, self).__init__()
        
        #Locally connected convolutiv layer
        self.conv1 = nn.Conv3d(in_channels = 1, out_channels = 32, kernel_size = 3, stride = 1, padding = 1)
        # stride : number of pixels by which we move the filter across the input image.
        # in channels : input data
        # out channels : number of unique features or patterns that a convolutional neural network (CNN) can learn and extract from an input image
        # padding : addition of extra pixels around the borders of the input images or feature map
        self.pool1 = nn.MaxPool3d(kernel_size = 2, stride = 2, padding = 0)
        self.bn1 = nn.BatchNorm3d(32)
        # Normalisation est de stabiliser et d'accélérer le processus d'apprentissage, en réduisant la sensibilité du réseau aux variations des valeurs d'entrée.
        
        #Fully connected layers
        self.fc1 = nn.Linear(32 * 32 * 32 * 32, 128)  # size of the volume
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, num_classes)
        
    def forward(self, x):# définit la manière dont les données passent à travers les différentes couches du réseau pour produire une sortie.
        x = self.pool1(self.bn1(torch.relu(self.conv1(x))))
        
        # Aplatir le tenseur pour les couches entièrement connectées
        x = x.view(-1, 32 * 32 * 32 * 32)  # redimensionne le tenseur
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x


In [5]:
class VolumeDataset(Dataset):
    def __init__(self, volumes, labels):
        self.volumes = volumes
        self.labels = labels
        
    def __len__(self):
        return len(self.volumes)
    
    def __getitem__(self, idx):
        volume = self.volumes[idx]
        label = self.labels[idx]
        return torch.tensor(volume, dtype=torch.float32), torch.tensor(label, dtype=torch.long)


In [7]:
def QuaternionLoss(q_pred, q_target):
        
        # Normalisation des quaternions (pour s'assurer qu'ils sont unitaires)
        q_pred = q_pred / torch.norm(q_pred, dim=-1, keepdim=True)
        q_target = q_target / torch.norm(q_target, dim=-1, keepdim=True)
        
        # Calcul du produit scalaire entre les quaternions prédits et cibles
        dot_product = torch.sum(q_pred * q_target, dim=-1)
        
        # Calcul de la perte : 1 - (dot_product ** 2)
        loss = 1.0 - dot_product ** 2
        
        # Moyenne de la perte pour le batch
        return loss.mean()

In [8]:
# Hyperparamètres
input_shape = (1, 64, 64, 64)  # Par exemple, volume de 64x64x64 avec 1 canal (grayscale)
num_classes = 6  # Par exemple, 6 orientations possibles
learning_rate = 0.001
num_epochs = 50

model = fly_alignement(num_classes = num_classes)


# Instancier la fonction de perte
criterion = QuaternionLoss(q_pred, q_target)

# Calculer la perte
loss = criterion(q_pred, q_target)

print(f'Loss: {loss.item()}')

optimizer = optim.Adam(model.parameters(), lr=learning_rate)


loss.backward()

NameError: name 'q_pred' is not defined